# **Enhanced Road Traffic Sign Detection with YOLOv5**

This Jupyter Notebook demonstrates the application of YOLOv5, a leading deep learning model, for object detection tasks, specifically focusing on a road sign dataset. YOLOv5 is celebrated for its efficiency and accuracy in identifying objects within images across various contexts. The primary aim to adapt, leverage YOLOv5 model's pre-existing knowledge and fine-tune it on the road sign dataset to enhance its detection capabilities to recognize and classify various road signs. This notebook encompasses the entire workflow, including data preparation, model configuration, training, and evaluation, offering an in-depth exploration of utilizing YOLOv5 for practical object detection challenges.This comprehensive approach aims to demonstrate the effectiveness of YOLOv5 in handling the nuanced task of road sign detection, a critical component for autonomous driving systems and traffic management.

## **Setting Up the YOLOv5 Environment**

In this section, we initialize our project environment by cloning the official YOLOv5 repository from GitHub. This step ensures we have the latest YOLOv5 framework and its dependencies. After cloning, we navigate into the yolov5 directory and reset the repository to a specific commit to ensure consistency and compatibility with our dataset and training procedure.

In [7]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

Cloning into 'yolov5'...
remote: Enumerating objects: 16413, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 16413 (delta 0), reused 0 (delta 0), pack-reused 16408
Receiving objects: 100% (16413/16413), 14.95 MiB | 26.71 MiB/s, done.
Resolving deltas: 100% (11260/11260), done.
/content/yolov5/yolov5
HEAD is now at 064365d8 Update parse_opt() in export.py to work as in train.py (#10789)


## **Importing Necessary Libraries and Modules**

Before diving into the model training and data processing, we import essential Python libraries and modules.

In [21]:
import glob
import yaml

import torch
from IPython.display import Image, display, clear_output
from google.colab import drive

from utils.downloads import attempt_download
from utils.plots import plot_results
from IPython.core.magic import register_line_cell_magic


## **Installing Dependencies and Verifying Setup**

This section focuses on installing all the Python packages required by YOLOv5 to function correctly, ensuring our environment is correctly set up. Check whether a CUDA-enabled GPU is available for training and inference tasks, providing a clear indication of the computational resources available to us (GPU or CPU)

In [ ]:
!pip install -qr requirements.txt

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## **Integrating Roboflow for Dataset Management**

In this step, we introduce Roboflow, a powerful tool for managing and versioning datasets, by installing its Python package. Roboflow facilitates the preprocessing, augmentation, and distribution of datasets, making it an invaluable resource for ML projects. After installing the Roboflow package, we initialize the Roboflow client with our API key. This connection allows us to access our specific project within Roboflow's workspace.

In [ ]:
!pip install -q roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR API KEY")
project = rf.workspace("sit-asmsw").project("road-sign-detection-in-real-time")
dataset = project.version(3).download("yolov5")

## **Loading the Dataset Configuration with YAML**
After setting the current working directory, where our YOLOv5 environment is prepared, we proceed to examine the dataset configuration. The YAML file, automatically generated by Roboflow, specifies paths to the training, validation, and test images, as well as the class names and number of classes.

This step ensures we have a clear overview of our dataset's structure and confirms that the dataset is correctly set up for the subsequent training process.

In [ ]:
%cd /content/yolov5

# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml


## **Defining the Number of Classes from the Dataset**

By reading the data.yaml file, which was previously confirmed to contain paths and class information, we retrieve the number of classes (nc) defined within. This is a critical step in dynamically adjusting our model to accurately reflect the dataset it will be trained on, ensuring consistency and correctness in the model's output layer to match the number of target classes.

This process not only automates the setup, making it more robust and less prone to manual errors but also enhances the notebook's adaptability to different datasets with varying numbers of classes.

In [ ]:
# define number of classes based on YAML
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

## **Reviewing the YOLOv5 Model Configuration**

Reviewing the model configuration helps ensure that the model aligns with our project's objectives and provides an opportunity to make any necessary adjustments, such as changing the number of filters in the final layer to match the number of classes in our dataset. This alignment is crucial for the model's ability to accurately detect and classify objects within our specific domain.

In [ ]:
#this is the model configuration
%cat /content/yolov5/models/yolov5s.yaml

## **Customizing IPython Magic for File Writing**

This function is particularly useful for customizing configuration files or other scripts based on the notebook's runtime state, such as updating model configuration files to reflect the number of classes dynamically determined from the dataset.

In [16]:
#customize iPython writefile
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## **Customizing YOLOv5 Model Configuration for Road Sign Detection**

This cell dynamically generates a custom YOLOv5 model configuration file named custom_yolov5s.yaml. This file is tailored specifically for the road sign detection project, taking into account the unique aspects of the dataset, such as the number of classes. By creating a custom model configuration, we ensure that the YOLOv5 model is optimally adjusted for our specific object detection task, potentially enhancing its effectiveness and efficiency.

- **Number of Classes (nc):** Set dynamically to match the number of road sign categories identified in the dataset.

- **Depth and Width Multiples:** Adjustments to the model's depth and width can fine-tune its size and computational complexity, affecting both accuracy and performance.

- **Anchors:** Predefined anchor box sizes that the model will use to detect objects. These are typically optimized for the scale and aspect ratios of objects in the training dataset.

- **Model Architecture:** The backbone and head of the YOLOv5 model are defined, detailing the layers and their configurations.

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## **Training YOLOv5 on Custom Road Sign Dataset**

In this section, we use the custom YOLOv5 configuration file tailored for our dataset, ensuring that the model architecture aligns with the specifics of our task, such as the number of classes.

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 80 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache


## **Navigating to the Training Output Directory**

In [ ]:
%cd runs/train/yolov5s_results
%cd /content/yolov5/runs/train/yolov5s_results/


## **Visualizing Training Results**

YOLOv5 automatically generates a results.png file that plots various training metrics over time, including precision, recall, and loss values. This visual representation helps in understanding how the model improved and stabilized over the training epochs.

In [ ]:
# plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

## **Reviewing Validation Batch Labels**

After the model has been trained, it's beneficial to visually inspect the validation results to understand how well the model is performing on unseen data. YOLOv5 generates images from the validation set with predicted bounding boxes and labels overlaid on the original images. This provides a qualitative assessment of the model's detection capabilities.

In [ ]:
Image(filename='/content/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

## **Displaying Augmented Training Data Example**

To gain insight into the training process and how the data is being presented to the model, it's useful to examine examples of the augmented training data.

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

## **Locating Trained Model Weights**

In [ ]:
# trained weights are saved by default in our weights folder
%ls /content/yolov5/runs/

## **Running Inference with Trained YOLOv5 Model**

After training the YOLOv5 model on the road sign dataset and identifying the location of the best performing weights (best.pt), the next step is to use these weights to run inference on new data.

In [ ]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/Road-Sign-Detection-in-Real-Time-3/test/images/


## **Displaying Inference Results**

After running inference on your test dataset, you might have a collection of images with detected objects highlighted by bounding boxes.

In [ ]:
for imageName in glob.glob('/content/yolov5/runs/detect/exp2/*.jpg')[:10]:
    display(Image(filename=imageName))


## **Saving Trained Model Weights to Google Drive**

In [ ]:
drive.mount('/content/gdrive')
%cp /content/yolov5/runs/train/yolov5s_results/weights/last.pt /content/gdrive/My\ Drive